In [1]:
from selenium import webdriver
import httpx
from bs4 import BeautifulSoup
import json
import copy

In [2]:
def get_html(url, element_selector):
    # Initialize the webdriver using WebDriverManager
    driver = webdriver.Chrome()
    driver.get(url)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # Find the element based on the provided selector
    selected_element = soup.select_one(element_selector)
    return selected_element

    
def extract_links_from_html(html_content):
    # Find all anchor tags (a) and extract their href attributes
    links = [a.get('href') for a in html_content.find_all('a')]

    # Filter out None values and remove duplicates
    unique_links = list(filter(None, set(links)))
    return unique_links


def extract_spell_info(html_content):
    spell_name_element = html_content.find('h2', class_='ak-spell-name')
    # Create a copy of the h2 element
    copy_spell_name_element = BeautifulSoup(str(spell_name_element), 'html.parser')
    # Remove undesired tags from the copy
    for tag in copy_spell_name_element.find_all(['div', 'span']):
        tag.decompose()
    # Extract spell name directly from the copied element
    spell_name = copy_spell_name_element.text.strip()
    # Extract range and AP content
    range_ap_content = html_content.find('span', class_='ak-spell-po-pa')
    # Create a copy of the h2 element
    copy_range_ap_element = BeautifulSoup(str(range_ap_content), 'html.parser')
    # Extract Minimum Required Level
    level =  html_content.select_one('.ak-spell-details-level .ak-selected')
    spell_description_content = html_content.find('span', class_='ak-spell-description')
    spell_details_effect_content = html_content.find('div', class_='ak-spell-details-effects')
    ak_main_content_elements = spell_details_effect_content.select('.ak-main-content')
    text = ""
    aside = ""
    effects = []
    for element in ak_main_content_elements:
        title = element.select_one('.ak-title').text.strip()
        if element.select_one('.ak-aside'):
            aside = element.select_one('.ak-aside').text.strip()
        if element.select_one('.ak-text'):
            text = element.select_one('.ak-text').text.strip()
        effects.append({
            "title": title,
            "text": aside + text
        })
        
    spell_details_effect_content = html_content.find('div', class_='ak-spell-details-effects')
    ak_main_content_elements = spell_details_effect_content.select('.ak-main-content')
    text = ""
    aside = ""
    effects = []
    for element in ak_main_content_elements:
        title = element.select_one('.ak-title').text.strip()
        if element.select_one('.ak-aside'):
            aside = element.select_one('.ak-aside').text.strip()
        if element.select_one('.ak-text'):
            text = element.select_one('.ak-text').text.strip()
        effects.append({
            "title": title,
            "text": aside + text
        })
    spell_details_critical_content = html_content.find('div', class_='ak-spell-details-critical')
    ak_main_content_elements = spell_details_critical_content.select('.ak-main-content')
    text = ""
    aside = ""
    critical = []
    for element in ak_main_content_elements:
        title = element.select_one('.ak-title').text.strip()
        if element.select_one('.ak-aside'):
            aside = element.select_one('.ak-aside').text.strip()
        if element.select_one('.ak-text'):
            text = element.select_one('.ak-text').text.strip()
        critical.append({
            "title": title,
            "text": aside + text
        })
    spell_details_other_content = html_content.find('div', class_='ak-spell-details-other')
    ak_main_content_elements = spell_details_other_content.select('.ak-main-content')
    text = ""
    aside = ""
    other = []
    for element in ak_main_content_elements:
        title = element.select_one('.ak-title').text.strip()
        if element.select_one('.ak-aside'):
            aside = element.select_one('.ak-aside').text.strip()
        if element.select_one('.ak-text'):
            text = element.select_one('.ak-text').text.strip()
        other.append({
            "title": title,
            "text": aside + text
        })
    return {
        "name" : spell_name,
        "range" : copy_range_ap_element.text.strip().split('/')[0].strip(),
        "ap": copy_range_ap_element.text.strip().split('/')[1].strip(),
        "description": spell_description_content.text.strip(),
        "level": level.text.strip(),
        "effects": effects,
        "critical": critical,
        "other": other
    }


def print_selected_spells(spell_list):
    spell_count_per_class = {}
    spell_details_list = []

    for spell_data in spell_list:
        class_name = spell_data["class_name"]
        spells = spell_data["spells"]

        for spell in spells:
            if any(
                effect["text"] == "Yes" and effect["title"] == "Modifiable range"
                for effect in spell["details"]["other"]
            ) and any(
                effect["text"] == "No" and effect["title"] == "Line of sight"
                for effect in spell["details"]["other"]
            ):
                # Append spell details to the list
                spell_details_list.append({
                    "class_name": class_name,
                    "spell_name": spell['details']['name'],
                    "description": spell['details']['description'],
                    "range": spell['details']['range'],
                    "level": spell['details']['level'],
                    "link": spell['spell_link']
                })

                # Update spell count for the class
                spell_count_per_class[class_name] = spell_count_per_class.get(class_name, 0) + 1

    # Sort classes by count in descending order
    sorted_classes = sorted(spell_count_per_class.items(), key=lambda x: x[1])

    for class_name, count in sorted_classes:
        print("---")
        print("---")
        print("---")
        for spell_details in spell_details_list:
            if spell_details["class_name"] == class_name:
                print(f"Class: {spell_details['class_name']}")
                print("Spell Details:")
                print(f"Name: {spell_details['spell_name']}")
                print(f"Description: {spell_details['description']}")
                print(f"Range: {spell_details['range']}")
                print(f"Level: {spell_details['level']}")
                print(f"Link: {spell_details['link']}")
                print("---")


    print("\nNumber of Spells Meeting Conditions per Class (Sorted by Count):")
    for class_name, count in sorted_classes:
        print(f"{class_name}: {count} spells")

In [ ]:
html_content = get_html('https://www.dofus.com/en/mmorpg/encyclopedia/classes', ".ak-content-sections")
character_links = extract_links_from_html(html_content)
data_list = []
for character_link in character_links:
    character_data = {'character_link': character_link, 'spells': []}
    html_content_character = get_html("https://www.dofus.com" + character_link, ".ak-spell-list-row")
    spell_links = extract_links_from_html(html_content_character)
    for spell_link in spell_links:
        html_content_spell = get_html("https://www.dofus.com" + spell_link, "main")
        if html_content_spell:
            # Assuming you want to store the spell link and corresponding HTML content
            spell_data = {'spell_link': "https://www.dofus.com/" + spell_link, 'html_content': html_content_spell}
            character_data['spells'].append(spell_data)
    data_list.append(character_data)

In [12]:
data_list_copy = copy.copy(data_list)
# Convert HTML content to strings outside the loop
for character_data in data_list_copy:
    character_data["character_link"] =  character_data["character_link"].replace("https://www.dofus.comhttps://www.dofus.com", "https://www.dofus.com")
    
    character_data['class_name'] = character_data["character_link"].split('-')[1].strip()
    for spell_data in character_data['spells']:
            html_content_str = str(spell_data['html_content'])
            spell_data['spell_link'] = spell_data['spell_link'].replace("https://www.dofus.comhttps://www.dofus.com", "https://www.dofus.com")
            spell_data['html_content'] = html_content_str
            spell_data['details'] = extract_spell_info( BeautifulSoup(spell_data['html_content'], 'html.parser'))
    # Sort spells by level
    character_data['spells'] = sorted(character_data['spells'], key=lambda x: int(x['details']['level']))
    # Save the copied data to a JSON file
    with open(character_data['class_name'] + '.json', 'w', encoding='utf-8') as json_file:
        json.dump(character_data, json_file, indent=2)
# Sort the entire data_list_copy by class_name
data_list_copy_sorted = sorted(data_list_copy, key=lambda x: x['class_name'])
# Save the copied and sorted data to a single JSON file
with open('dofus_spells.json', 'w', encoding='utf-8') as json_file:
    json.dump(data_list_copy_sorted, json_file, indent=2)

In [8]:
# Specify the path to your JSON file
json_file_path = 'dofus_spells.json'

# Load data from the JSON file
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    dofus_spells_data = json.load(json_file)
    
print_selected_spells(dofus_spells_data)

---
---
---
Class: sadida
Spell Details:
Name: Tear
Description: Inflicts Water damage.

On one of the caster's Trees: transforms it into a Leafy Tree.

On an Infected enemy: inflicts Water damage on all Infected enemies.
Range: 1 - 4 Range
Level: 10
Link: https://www.dofus.comhttps://www.dofus.com//en/mmorpg/encyclopedia/spells/details?id=13528&level=42112&selector=1&characterlevel=1&showlevel=0&startingspell=0
---
---
---
---
Class: feca
Spell Details:
Name: Barricade
Description: Makes the targeted ally Invulnerable in Close Combat.
For 1 turn, if they receive ranged damage, increases their MP (stackable 3 times).

The MP increase is not applied if the target is under the effects of the Bastion spell.

On a free cell: summons a static shield that makes adjacent allies in its glyph-aura Invulnerable in Close Combat.
This shield is destroyed on the caster's next turn.
Range: 4 Range
Level: 10
Link: https://www.dofus.comhttps://www.dofus.com//en/mmorpg/encyclopedia/spells/details?id=12